In [1]:
import pyspark

In [2]:
import findspark

In [3]:
findspark.init()

In [4]:
#!pip install py4j==0.10.9.5

In [5]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg

spark = (SparkSession
        .builder
        .appName('AuthorsAges')
        .getOrCreate())

In [7]:
data_df = spark.createDataFrame([('Brooke',20), ('Denny',31), ('Jules',30),("TD",35),('Brooke',25)], ['name','age'])

avg_df = data_df.groupBy('name').agg(avg('age'))
avg_df.show()

+------+--------+
|  name|avg(age)|
+------+--------+
|Brooke|    22.5|
| Denny|    31.0|
| Jules|    30.0|
|    TD|    35.0|
+------+--------+



# 스키마 정의

## 프로그래밍 스타일로 정의

In [8]:
from pyspark.sql.types import *

schema = StructType([StructField("author", StringType(), False),
                    StructField("title", StringType(), False)])

In [9]:
schema

StructType([StructField('author', StringType(), False), StructField('title', StringType(), False)])

## DDL 사용하여 정의

In [10]:
schema = "author STRING, title STRING, pages INT"

# p.52 예제

In [11]:
from pyspark.sql import SparkSession

# DDL 사용하여 스키마 정의
#schema = "'ID' INT, 'First' STRING, 'Last' STRING, 'Url' STRING, 'Published' STRING, 'HITS' INT, 'Campaigns' ARRAY<STRING>"

schema = StructType([
   StructField("Id", IntegerType(), False),
   StructField("First", StringType(), False),
   StructField("Last", StringType(), False),
   StructField("Url", StringType(), False),
   StructField("Published", StringType(), False),
   StructField("Hits", IntegerType(), False),
   StructField("Campaigns", ArrayType(StringType()), False)])

data = [[1, "Jules", "Damji", "https://tinyurl.1", "1/4/2016", 4535, ["twitter", "LinkedIn"]],
       [2, "Brooke","Wenig","https://tinyurl.2", "5/5/2018", 8908, ["twitter", "LinkedIn"]],
       [3, "Denny", "Lee", "https://tinyurl.3","6/7/2019",7659, ["web", "twitter", "FB", "LinkedIn"]],
       [4, "Tathagata", "Das","https://tinyurl.4", "5/12/2018", 10568, ["twitter", "FB"]],
       [5, "Matei","Zaharia", "https://tinyurl.5", "5/14/2014", 40578, ["web", "twitter", "FB", "LinkedIn"]],
       [6, "Reynold", "Xin", "https://tinyurl.6", "3/2/2015", 25568, ["twitter", "LinkedIn"]]
      ]

# 메인 프로그램

#스파크세션 생성
spark = (SparkSession
        .builder
        .appName("Example-3_6")
        .getOrCreate())

blogs_df = spark.createDataFrame(data, schema)
blogs_df.show()

+---+---------+-------+-----------------+---------+-----+--------------------+
| Id|    First|   Last|              Url|Published| Hits|           Campaigns|
+---+---------+-------+-----------------+---------+-----+--------------------+
|  1|    Jules|  Damji|https://tinyurl.1| 1/4/2016| 4535| [twitter, LinkedIn]|
|  2|   Brooke|  Wenig|https://tinyurl.2| 5/5/2018| 8908| [twitter, LinkedIn]|
|  3|    Denny|    Lee|https://tinyurl.3| 6/7/2019| 7659|[web, twitter, FB...|
|  4|Tathagata|    Das|https://tinyurl.4|5/12/2018|10568|       [twitter, FB]|
|  5|    Matei|Zaharia|https://tinyurl.5|5/14/2014|40578|[web, twitter, FB...|
|  6|  Reynold|    Xin|https://tinyurl.6| 3/2/2015|25568| [twitter, LinkedIn]|
+---+---------+-------+-----------------+---------+-----+--------------------+



In [12]:
blogs_df.printSchema()

root
 |-- Id: integer (nullable = false)
 |-- First: string (nullable = false)
 |-- Last: string (nullable = false)
 |-- Url: string (nullable = false)
 |-- Published: string (nullable = false)
 |-- Hits: integer (nullable = false)
 |-- Campaigns: array (nullable = false)
 |    |-- element: string (containsNull = true)



# 로우 (p.58~)

In [13]:
from pyspark.sql import Row

In [14]:
blog_row = Row(6, "Reynold", "Xin", "https://tinyurl.6", 255568, "3/2/2015", ["twitter", "LinkedIn"])

In [15]:
blog_row[1]

'Reynold'

In [16]:
rows = [Row("Matei Zaharia", "CA"), Row("Reynold Xin", "CA")]
author_df = spark.createDataFrame(rows, ["Authors", "State"])
author_df.show()

+-------------+-----+
|      Authors|State|
+-------------+-----+
|Matei Zaharia|   CA|
|  Reynold Xin|   CA|
+-------------+-----+



# DataFrameReader/DataFrameWriter 사용하기(p.60~)

In [18]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.appName("Chapter03").getOrCreate()

# 스키마 정의
fire_schema = StructType([StructField('CallNumber', IntegerType(), True),
                         StructField('UnitID', StringType(), True),
                         StructField('IncidentNumber', IntegerType(), True),
                         StructField('CallType', StringType(), True),
                         StructField('CallDate', StringType(), True),
                         StructField('WatchDate', StringType(), True),
                         StructField('CallFinalDisposition', StringType(), True),
                         StructField('AvailableDtTm', StringType(), True), 
                         StructField('Address', StringType(), True),
                         StructField('City', StringType(), True), 
                         StructField('Zipcode', StringType(), True), 
                         StructField('Battalion', StringType(), True),
                         StructField('StationArea', StringType(), True), 
                         StructField('Box', StringType(), True),
                         StructField('OriginalPriority', StringType(), True),
                         StructField('Priority', StringType(), True), 
                         StructField('FinalPriority', IntegerType(), True), 
                         StructField('ASLUnit', BooleanType(), True), 
                         StructField('CalltypeGroup', StringType(), True), 
                         StructField('NumAlarms', IntegerType(), True),
                         StructField('UnitType', StringType(), True), 
                         StructField('UnitSequenceInCallDistrict', IntegerType(), True), 
                         StructField('FirePreventionDistirct', StringType(), True),
                         StructField('SupervisiorDistrict', StringType(), True),
                         StructField('Neighborhood', StringType(), True),
                         StructField('Location', StringType(), True), 
                         StructField('RowID', StringType(), True),
                         StructField('Delay', FloatType(), True) 
                         ])

sf_fire_file = "C:/0. Modeling/1. Modeling Code/Pyspark/pyspark_dataset/LearningSparkV2-master/chapter3/data/sf-fire-calls.csv"
fire_df = spark.read.csv(sf_fire_file, header = True, schema = fire_schema)

In [19]:
fire_df.show(5, False)

+----------+------+--------------+----------------+----------+----------+--------------------+----------------------+---------------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------+--------------------------+----------------------+-------------------+---------------------+-------------------------------------+-------------+---------+
|CallNumber|UnitID|IncidentNumber|CallType        |CallDate  |WatchDate |CallFinalDisposition|AvailableDtTm         |Address                    |City|Zipcode|Battalion|StationArea|Box |OriginalPriority|Priority|FinalPriority|ASLUnit|CalltypeGroup|NumAlarms|UnitType|UnitSequenceInCallDistrict|FirePreventionDistirct|SupervisiorDistrict|Neighborhood         |Location                             |RowID        |Delay    |
+----------+------+--------------+----------------+----------+----------+--------------------+----------------------+---------------------------+----+-------+

## 데이터 프레임을 파케이 파일/ SQL 테이블로 저장하기

In [20]:
print(sys.version)

3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]


In [21]:
spark.version

'3.3.2'

### 파케이 파일로 저장

In [22]:
parquet_path = "C:/0. Modeling/1. Modeling Code/Pyspark/pyspark_dataset/LearningSparkV2-master/chapter3/data/fire_df.parquet"
fire_df.write.format("parquet").mode('overwrite').save(parquet_path)

### SQL 테이블로 저장 (오류)

parquet_table = "C:/0. Modeling/1. Modeling Code/Pyspark/pyspark_dataset/LearningSparkV2-master/chapter3/data/few_fire_tbl"
fire_df.write.format("parquet").mode('overwrite').saveAsTable(parquet_table)

## 프로젝션과 필터 (p.63~)

In [25]:
few_fire_df = (fire_df
              .select("IncidentNumber", "AvailableDtTm","CallType")
              .where(col("CallType") != "Medical Incident"))

few_fire_df.show(5, truncate = False)

+--------------+----------------------+--------------+
|IncidentNumber|AvailableDtTm         |CallType      |
+--------------+----------------------+--------------+
|2003235       |01/11/2002 01:51:44 AM|Structure Fire|
|2003250       |01/11/2002 04:16:46 AM|Vehicle Fire  |
|2003259       |01/11/2002 06:01:58 AM|Alarms        |
|2003279       |01/11/2002 08:03:26 AM|Structure Fire|
|2003301       |01/11/2002 09:46:44 AM|Alarms        |
+--------------+----------------------+--------------+
only showing top 5 rows



In [26]:
### 신고로 기록된 Calltype 종류

from pyspark.sql.functions import *

(fire_df
    .select("CallType")
    .where(col("CallType").isNotNull())
    .agg(countDistinct("CallType").alias("DistinctCallTypes"))
    .show()
)

+-----------------+
|DistinctCallTypes|
+-----------------+
|               30|
+-----------------+



In [27]:
### null이 아닌 신고 타입의 목록

(fire_df
    .select("CallType")
    .where(col("CallType").isNotNull())
    .distinct()
    .show(10, False)
)

+-----------------------------+
|CallType                     |
+-----------------------------+
|Elevator / Escalator Rescue  |
|Aircraft Emergency           |
|Alarms                       |
|Odor (Strange / Unknown)     |
|Citizen Assist / Service Call|
|HazMat                       |
|Explosion                    |
|Oil Spill                    |
|Vehicle Fire                 |
|Suspicious Package           |
+-----------------------------+
only showing top 10 rows



## 칼럼 이름 변경

In [28]:
new_fire_df = fire_df.withColumnRenamed("Delay","ResponseDelayedinMins")

(new_fire_df
    .select("ResponseDelayedinMins")
    .where(col("ResponseDelayedinMins")>5)
    .show(5, False)
)

+---------------------+
|ResponseDelayedinMins|
+---------------------+
|5.35                 |
|6.25                 |
|5.2                  |
|5.6                  |
|7.25                 |
+---------------------+
only showing top 5 rows



## 칼럼 속성 변경

In [29]:
fire_ts_df= (new_fire_df
             .withColumn("IncidentDate", to_timestamp(col("CallDate"), "MM/dd/yyyy"))
             .drop("CallDate")
             .withColumn("OnWatchDate", to_timestamp(col("WatchDate"), "MM/dd/yyyy"))
             .drop("WatchDate")
             .withColumn("AvailableDtTS", to_timestamp(col("AvailableDtTm"), "MM/dd/yyyy hh:mm:ss a"))
             .drop("AvailableDtTm") 
            )

In [30]:
fire_ts_df.show(5,False)

+----------+------+--------------+----------------+--------------------+---------------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------+--------------------------+----------------------+-------------------+---------------------+-------------------------------------+-------------+---------------------+-------------------+-------------------+-------------------+
|CallNumber|UnitID|IncidentNumber|CallType        |CallFinalDisposition|Address                    |City|Zipcode|Battalion|StationArea|Box |OriginalPriority|Priority|FinalPriority|ASLUnit|CalltypeGroup|NumAlarms|UnitType|UnitSequenceInCallDistrict|FirePreventionDistirct|SupervisiorDistrict|Neighborhood         |Location                             |RowID        |ResponseDelayedinMins|IncidentDate       |OnWatchDate        |AvailableDtTS      |
+----------+------+--------------+----------------+--------------------+---------------------------+----

In [31]:
(fire_ts_df
    .select("IncidentDate", "OnWatchDate","AvailableDtTS")
    .show(5, False))

+-------------------+-------------------+-------------------+
|IncidentDate       |OnWatchDate        |AvailableDtTS      |
+-------------------+-------------------+-------------------+
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 01:51:44|
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 03:01:18|
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 02:39:50|
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 04:16:46|
|2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 06:01:58|
+-------------------+-------------------+-------------------+
only showing top 5 rows



In [32]:
(fire_ts_df
    .select(year("IncidentDate"))
    .distinct()
    .orderBy(year("IncidentDate"))
    .show()
)

+------------------+
|year(IncidentDate)|
+------------------+
|              2000|
|              2001|
|              2002|
|              2003|
|              2004|
|              2005|
|              2006|
|              2007|
|              2008|
|              2009|
|              2010|
|              2011|
|              2012|
|              2013|
|              2014|
|              2015|
|              2016|
|              2017|
|              2018|
+------------------+



## 집계 연산

In [33]:
(fire_ts_df
    .select("CallType")
    .where(col('CallType').isNotNull())
    .groupby("CallType")
    .count()
    .orderBy("count", ascending = False)
    .show(n=10, truncate = False)
)

+-------------------------------+------+
|CallType                       |count |
+-------------------------------+------+
|Medical Incident               |113794|
|Structure Fire                 |23319 |
|Alarms                         |19406 |
|Traffic Collision              |7013  |
|Citizen Assist / Service Call  |2524  |
|Other                          |2166  |
|Outside Fire                   |2094  |
|Vehicle Fire                   |854   |
|Gas Leak (Natural and LP Gases)|764   |
|Water Rescue                   |755   |
+-------------------------------+------+
only showing top 10 rows



In [34]:
import pyspark.sql.functions as F

(fire_ts_df
    .select(F.sum("NumAlarms"), F.avg("ResponseDelayedinMins"),
           F.min("ResponseDelayedinMins"), F.max("ResponseDelayedinMins"))
    .show()
)

+--------------+--------------------------+--------------------------+--------------------------+
|sum(NumAlarms)|avg(ResponseDelayedinMins)|min(ResponseDelayedinMins)|max(ResponseDelayedinMins)|
+--------------+--------------------------+--------------------------+--------------------------+
|        176170|         3.892364154521585|               0.016666668|                   1844.55|
+--------------+--------------------------+--------------------------+--------------------------+



# 데이터세트 API

## 정적 타입 객체, 동적 타입 객체, 포괄적인 Row

In [35]:
row = Row(350, True, "Learning Spark 2E", None)

In [36]:
row[2]

'Learning Spark 2E'

# 카탈리스트 옵티마이저

In [38]:
from pyspark.sql import SparkSession

spark = (SparkSession
        .builder
        .appName("PythonMnMCount")
        .getOrCreate())

mnm_file = 'C:/0. Modeling/1. Modeling Code/Pyspark/pyspark_dataset/mnm_dataset.csv'

mnm_df = (spark.read.format("csv")
         .option("header","true")
         .option("inferSchema","true")
         .load(mnm_file))

In [39]:
mnm_df

DataFrame[State: string, Color: string, Count: int]

In [40]:
count_mnm_df = (mnm_df
               .select("State","Color","Count")
               .groupBy("State","Color")
               .agg(count("Count")
               .alias("Total"))
               .orderBy("Total", ascending = True))

In [41]:
# 파이썬 코드가 거치게 되는 다른 스테이지들을 확인

count_mnm_df.explain(True)

== Parsed Logical Plan ==
'Sort ['Total ASC NULLS FIRST], true
+- Aggregate [State#909, Color#910], [State#909, Color#910, count(Count#911) AS Total#923L]
   +- Project [State#909, Color#910, Count#911]
      +- Relation [State#909,Color#910,Count#911] csv

== Analyzed Logical Plan ==
State: string, Color: string, Total: bigint
Sort [Total#923L ASC NULLS FIRST], true
+- Aggregate [State#909, Color#910], [State#909, Color#910, count(Count#911) AS Total#923L]
   +- Project [State#909, Color#910, Count#911]
      +- Relation [State#909,Color#910,Count#911] csv

== Optimized Logical Plan ==
Sort [Total#923L ASC NULLS FIRST], true
+- Aggregate [State#909, Color#910], [State#909, Color#910, count(Count#911) AS Total#923L]
   +- Relation [State#909,Color#910,Count#911] csv

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [Total#923L ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(Total#923L ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [plan_id=471]
      +- HashAggre